# QMitigate: Comprehensive Documentation Guide

<div align='center'>

**High-Performance Quantum Error Mitigation Engine**

*A C++ quantum state-vector simulator with Zero-Noise Extrapolation for NISQ processors*

</div>

---

## Table of Contents

1. [Introduction & Motivation](#1.-Introduction-&-Motivation)
2. [Architecture Overview](#2.-Architecture-Overview)
3. [Quantum States](#3.-Quantum-States)
4. [Quantum Gates](#4.-Quantum-Gates)
5. [Circuit Building & Simulation](#5.-Circuit-Building-&-Simulation)
6. [Noise Modeling](#6.-Noise-Modeling)
7. [Zero-Noise Extrapolation (ZNE)](#7.-Zero-Noise-Extrapolation)
8. [Bias-Variance Analysis](#8.-Bias-Variance-Analysis)
9. [Performance Benchmarks](#9.-Performance-Benchmarks)
10. [Advanced Examples](#10.-Advanced-Examples)
11. [API Reference Summary](#11.-API-Reference-Summary)


## 1. Introduction & Motivation

### The NISQ Challenge

Quantum computers today operate in the **Noisy Intermediate-Scale Quantum (NISQ)** era:

| Challenge | Typical Value | Impact |
|-----------|--------------|--------|
| Gate error rate | 0.1–1% per gate | Limits circuit depth |
| Decoherence (T1/T2) | 50–100 μs | Time window for computation |
| Qubit count | 50–1000 | Not enough for full error correction |

**QMitigate** addresses this by implementing **Zero-Noise Extrapolation (ZNE)**, an error mitigation
technique that recovers accurate results from noisy quantum hardware *without* requiring additional qubits.

### Key Research References

1. Temme, K., Bravyi, S., & Gambetta, J. M. (2017). *"Error mitigation for short-depth quantum circuits."* PRL.
2. Giurgica-Tiron, T., et al. (2020). *"Digital zero noise extrapolation for quantum error mitigation."* IEEE ICQC.
3. LaRose, R., et al. (2022). *"Mitiq: A software package for error mitigation."* Quantum.


### Setup
Install dependencies and import the QMitigate library.

In [ ]:
import sys
sys.path.insert(0, '../python')

from qmitigate import (
    Circuit, Simulator, QuantumState,
    create_depolarizing_noise_model,
    create_bell_state, create_ghz_state, create_w_state,
    fold_circuit_global,
    ZeroNoiseExtrapolator,
    richardson_extrapolate, linear_extrapolate, exponential_extrapolate,
    run_benchmark,
)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch
import matplotlib.gridspec as gridspec

# Plotting defaults
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams.update({
    'figure.figsize': (12, 6),
    'font.size': 12,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
})

print('✅ QMitigate loaded successfully')


## 2. Architecture Overview

QMitigate is structured as a high-performance **C++ core** with **Python bindings** via PyBind11:

```
┌─────────────────────────────────────────────────────┐
│             Python Layer (Research API)              │
│  ┌──────────────┐  ┌────────────┐  ┌────────────┐  │
│  │   ZNE Engine  │  │ Benchmarks │  │ Notebooks  │  │
│  └──────┬───────┘  └─────┬──────┘  └────────────┘  │
├─────────┼────────────────┼──────────────────────────┤
│         │   PyBind11     │                          │
├─────────┼────────────────┼──────────────────────────┤
│              C++ Core (Performance)                  │
│  ┌──────────┐ ┌───────┐ ┌───────────┐ ┌─────────┐  │
│  │ Simulator│ │ Gates │ │NoiseModel │ │ QState  │  │
│  └──────────┘ └───────┘ └───────────┘ └─────────┘  │
│                  OpenMP Parallelism                  │
└─────────────────────────────────────────────────────┘
```

### Design Decisions

| Decision | Rationale |
|----------|----------|
| **State-vector simulation** | Exact amplitudes needed for ZNE research |
| **C++ with OpenMP** | 50–100× faster than pure Python |
| **PyBind11** | Zero-copy data sharing, numpy compatibility |
| **Sparse gate application** | O(2ⁿ) vs O(4ⁿ) memory per gate |


## 3. Quantum States

A pure quantum state of $N$ qubits is a complex vector in a $2^N$-dimensional Hilbert space:

$$|\psi\rangle = \sum_{i=0}^{2^N - 1} \alpha_i |i\rangle, \quad \sum_i |\alpha_i|^2 = 1$$

### 3.1 Factory States

QMitigate provides factory functions for common quantum states:


In [ ]:
# Bell State: (|00⟩ + |11⟩) / √2
bell = create_bell_state()
print('Bell State |Φ+⟩:')
print(f'  Qubits: {bell.num_qubits()}')
print(f'  Amplitudes: |00⟩={bell.amplitude(0):.4f}, |01⟩={bell.amplitude(1):.4f}, '
      f'|10⟩={bell.amplitude(2):.4f}, |11⟩={bell.amplitude(3):.4f}')
print(f'  Is normalized: {bell.is_normalized()}')
print(f'  ⟨Z₀⟩ = {bell.expectation_Z(0):.4f} (expected: 0.0)')
print()

# GHZ State: (|000⟩ + |111⟩) / √2
ghz = create_ghz_state(3)
print(f'GHZ State (3 qubits): {ghz.num_qubits()} qubits, dim = {ghz.dimension()}')
print(f'  P(|000⟩) = {ghz.probability(0):.4f}')
print(f'  P(|111⟩) = {ghz.probability(7):.4f}')
print()

# W State: (|100⟩ + |010⟩ + |001⟩) / √3
w = create_w_state(3)
print(f'W State (3 qubits):')
probs = w.get_probabilities()
for i, p in enumerate(probs):
    if p > 0.01:
        print(f'  P(|{i:03b}⟩) = {p:.4f}')


### 3.2 State Properties & Fidelity

The fidelity between two pure states measures their overlap:
$$F(|\psi\rangle, |\phi\rangle) = |\langle\psi|\phi\rangle|^2$$


In [ ]:
# Fidelity between identical states
bell2 = create_bell_state()
print(f'Fidelity(Bell, Bell) = {bell.fidelity(bell2):.4f} (expected: 1.0)')

# Fidelity between orthogonal states
state_0 = QuantumState(1)  # |0⟩
print(f'Norm of |0⟩ = {state_0.norm():.4f}')
print(f'P(|0⟩) = {state_0.probability(0):.4f}')
print(f'P(|1⟩) = {state_0.probability(1):.4f}')


## 4. Quantum Gates

QMitigate implements a comprehensive gate set with O(2ⁿ)-sparse application.

### 4.1 Supported Gates

| Category | Gates | Description |
|----------|-------|-------------|
| **Single-qubit** | X, Y, Z, H, S, T, S†, T† | Pauli, Hadamard, phase gates |
| **Rotation** | RX(θ), RY(θ), RZ(θ), U3(θ,φ,λ) | Parameterized rotations |
| **Two-qubit** | CNOT, CZ, CY, SWAP | Entangling gates |
| **Three-qubit** | Toffoli (CCX), Fredkin (CSWAP) | Multi-controlled gates |

### 4.2 Gate Demonstrations


In [ ]:
# Demonstrate single-qubit gates
def show_gate_effect(gate_name, gate_fn, initial='|0⟩'):
    """Apply a gate and show the resulting state."""
    circuit = Circuit(1)
    gate_fn(circuit)
    sim = Simulator()
    state = sim.run(circuit)
    probs = state.get_probabilities()
    print(f'{gate_name:.<12} {initial} → P(|0⟩)={probs[0]:.3f}, P(|1⟩)={probs[1]:.3f}, '
          f'⟨Z⟩={state.expectation_Z(0):+.3f}')

print('Single-Qubit Gate Effects on |0⟩:')
print('=' * 60)
show_gate_effect('Hadamard (H)', lambda c: c.h(0))
show_gate_effect('Pauli-X',      lambda c: c.x(0))
show_gate_effect('Pauli-Y',      lambda c: c.y(0))
show_gate_effect('Pauli-Z',      lambda c: c.z(0))
show_gate_effect('RX(π/2)',      lambda c: c.rx(0, np.pi/2))
show_gate_effect('RY(π/2)',      lambda c: c.ry(0, np.pi/2))
show_gate_effect('RZ(π/2)',      lambda c: c.rz(0, np.pi/2))


### 4.3 Entangling Gates: Building a Bell State Step-by-Step


In [ ]:
# Build Bell state manually: H on q0, then CNOT(0→1)
circuit = Circuit(2)
circuit.h(0)     # |00⟩ → (|00⟩ + |10⟩)/√2
circuit.cnot(0, 1)  # → (|00⟩ + |11⟩)/√2

sim = Simulator()
state = sim.run(circuit)

print('Bell State via Circuit:')
print(f'  Circuit depth: {circuit.depth()} gates')
probs = state.get_probabilities()
labels = ['00', '01', '10', '11']
for label, p in zip(labels, probs):
    if p > 0.001:
        print(f'  P(|{label}⟩) = {p:.4f}')

# Verify with factory function
factory_bell = create_bell_state()
print(f'\n  Fidelity with factory Bell state: {state.fidelity(factory_bell):.6f}')


## 5. Circuit Building & Simulation

### 5.1 VQE-Style Ansatz Circuit

Variational Quantum Eigensolver (VQE) circuits alternate between entangling layers
and parameterized rotations:


In [ ]:
def create_vqe_ansatz(num_qubits, depth, params=None):
    """Create a hardware-efficient VQE ansatz."""
    circuit = Circuit(num_qubits)
    rng = np.random.default_rng(42)
    
    # Initial superposition
    for q in range(num_qubits):
        circuit.h(q)
    
    # Alternating entangling + rotation layers
    for layer in range(depth):
        # Entangling layer (linear connectivity)
        for q in range(num_qubits - 1):
            circuit.cnot(q, q + 1)
        # Rotation layer
        for q in range(num_qubits):
            angle = params[layer * num_qubits + q] if params else 0.1 * (layer + 1)
            circuit.rz(q, angle)
            circuit.ry(q, angle * 0.5)
    
    return circuit

# Create and run
vqe_circuit = create_vqe_ansatz(num_qubits=4, depth=3)
sim = Simulator()
result = sim.run(vqe_circuit)

print(f'VQE Ansatz: {vqe_circuit.num_qubits} qubits, {vqe_circuit.depth()} gates')
print(f'Expectation values:')
for q in range(4):
    print(f'  ⟨Z_{q}⟩ = {result.expectation_Z(q):.4f}')


## 6. Noise Modeling

### 6.1 The Depolarizing Channel

The primary noise model in QMitigate is the **depolarizing channel**:

$$\mathcal{E}(\rho) = (1-p)\rho + \frac{p}{3}(X\rho X + Y\rho Y + Z\rho Z)$$

where $p$ is the error probability per gate. This is the most common model for
superconducting qubit systems.

### 6.2 Impact of Noise on Circuit Results


In [ ]:
# Compare ideal vs noisy execution
circuit = create_vqe_ansatz(num_qubits=3, depth=5)

# Ideal
ideal_sim = Simulator()
ideal_state = ideal_sim.run(circuit)
ideal_z0 = ideal_state.expectation_Z(0)

# Sweep error rates
error_rates = [0.001, 0.005, 0.01, 0.02, 0.05, 0.1]
noisy_means = []
noisy_stds = []

for rate in error_rates:
    noise = create_depolarizing_noise_model(rate)
    noisy_sim = Simulator(noise)
    vals = [noisy_sim.run(circuit).expectation_Z(0) for _ in range(50)]
    noisy_means.append(np.mean(vals))
    noisy_stds.append(np.std(vals))

# Plot
fig, ax = plt.subplots(figsize=(10, 5))
ax.axhline(y=ideal_z0, color='#2ecc71', linewidth=2.5, linestyle='--', label=f'Ideal ⟨Z₀⟩ = {ideal_z0:.3f}')
ax.errorbar(error_rates, noisy_means, yerr=noisy_stds, fmt='o-', color='#e74c3c',
            capsize=5, linewidth=2, markersize=8, label='Noisy ⟨Z₀⟩')
ax.set_xlabel('Gate Error Rate (p)')
ax.set_ylabel('⟨Z₀⟩ Expectation Value')
ax.set_title('Impact of Depolarizing Noise on Expectation Values', fontweight='bold')
ax.set_xscale('log')
ax.legend(fontsize=11)
plt.tight_layout()
plt.show()


## 7. Zero-Noise Extrapolation (ZNE)

### 7.1 Key Insight

If we can **scale the noise level** $\lambda$, we can measure results at multiple scales
and **extrapolate back to zero noise**:

$$E(\lambda) = E(0) + a_1\lambda + a_2\lambda^2 + \ldots$$

### 7.2 Digital Folding

To scale noise without hardware changes, we apply **digital folding**:

$$U \xrightarrow{3\times} U U^\dagger U \xrightarrow{5\times} U U^\dagger U U^\dagger U$$

Since $U^\dagger U = I$, the logical operation is unchanged, but the physical circuit depth
is tripled/quintupled, amplifying the noise.

### 7.3 Step-by-Step ZNE Demonstration


In [ ]:
# Step 1: Create circuit and get ideal value
circuit = create_vqe_ansatz(num_qubits=3, depth=5)
ideal_sim = Simulator()
ideal_state = ideal_sim.run(circuit)
ideal_exp_z = ideal_state.expectation_Z(0)
print(f'Step 1 - Ideal ⟨Z₀⟩ = {ideal_exp_z:.4f}')

# Step 2: Create noisy simulator
ERROR_RATE = 0.02
noise = create_depolarizing_noise_model(ERROR_RATE)
noisy_sim = Simulator(noise)

# Step 3: Measure at multiple noise scales via digital folding
scale_factors = [1, 3, 5, 7]
expectations_at_scales = []

for scale in scale_factors:
    folded = fold_circuit_global(circuit, scale)
    vals = [noisy_sim.run(folded).expectation_Z(0) for _ in range(100)]
    mean_val = np.mean(vals)
    expectations_at_scales.append(mean_val)
    print(f'Step 3 - Scale {scale}×: ⟨Z₀⟩ = {mean_val:.4f} (depth: {folded.depth()})')

# Step 4: Extrapolate to zero noise
mitigated_richardson = richardson_extrapolate(
    [float(s) for s in scale_factors], expectations_at_scales)
mitigated_linear, slope = linear_extrapolate(
    [float(s) for s in scale_factors], expectations_at_scales)
mitigated_exp, exp_params = exponential_extrapolate(
    [float(s) for s in scale_factors], expectations_at_scales)

print(f'\n{"="*55}')
print(f'{"Method":<20} {"Estimate":>10} {"Error":>10} {"Improvement":>12}')
print(f'{"="*55}')
raw_err = abs(ideal_exp_z - expectations_at_scales[0])
for name, val in [('Raw (noisy)', expectations_at_scales[0]),
                   ('Richardson', mitigated_richardson),
                   ('Linear', mitigated_linear),
                   ('Exponential', mitigated_exp)]:
    err = abs(ideal_exp_z - val)
    imp = raw_err / max(err, 1e-10)
    print(f'{name:<20} {val:>10.4f} {err:>10.4f} {imp:>10.1f}×')


### 7.4 The 'Money Plot': ZNE Recovery Visualization


In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))

# Noisy data points
ax.scatter(scale_factors, expectations_at_scales,
           s=150, c='#e74c3c', marker='o', label='Noisy Measurements', zorder=5)

# Polynomial fit curve
x_fit = np.linspace(0, max(scale_factors), 200)
coeffs = np.polyfit(scale_factors, expectations_at_scales, len(scale_factors) - 1)
y_fit = np.polyval(coeffs, x_fit)
ax.plot(x_fit, y_fit, 'b--', linewidth=2, alpha=0.7, label='Richardson Polynomial Fit')

# Ideal value
ax.axhline(y=ideal_exp_z, color='#2ecc71', linewidth=2.5,
           linestyle='-', label=f'Ideal Value ({ideal_exp_z:.3f})')

# Extrapolated points
markers = [('*', '#9b59b6', 'Richardson'), ('D', '#3498db', 'Linear'), ('s', '#e67e22', 'Exponential')]
values = [mitigated_richardson, mitigated_linear, mitigated_exp]
for (m, c, name), val in zip(markers, values):
    ax.scatter([0], [val], s=250, c=c, marker=m, zorder=10,
              edgecolors='white', linewidth=2, label=f'{name} ({val:.3f})')

ax.set_xlabel('Noise Scale Factor (λ)', fontsize=14)
ax.set_ylabel('Expectation Value ⟨Z⟩', fontsize=14)
ax.set_title('Zero-Noise Extrapolation: Recovering Accurate Quantum Results',
             fontsize=14, fontweight='bold')
ax.legend(loc='lower left', fontsize=10)
ax.set_xlim(-0.5, max(scale_factors) + 0.5)
plt.tight_layout()
plt.savefig('zne_recovery_comparison.png', dpi=150, bbox_inches='tight')
plt.show()
print('✓ Saved: zne_recovery_comparison.png')


### 7.5 High-Level ZNE API

The `ZeroNoiseExtrapolator` class provides a convenient interface:


In [ ]:
zne = ZeroNoiseExtrapolator(
    simulator=noisy_sim,
    scale_factors=[1, 3, 5],
    extrapolation_method='richardson',
    shots_per_scale=50
)

mitigated, raw_data = zne.mitigate_expectation_Z(circuit, qubit=0, return_raw=True)

print(f'ZeroNoiseExtrapolator Results:')
print(f'  Ideal:     {ideal_exp_z:.4f}')
print(f'  Noisy:     {raw_data["unmitigated"]:.4f}')
print(f'  Mitigated: {mitigated:.4f}')
print(f'  Improved:  {raw_data["improvement"]}')


## 8. Bias-Variance Analysis

ZNE reduces **bias** (systematic error) but can increase **variance** (statistical fluctuations).
This is the fundamental tradeoff described in the Mitiq paper (LaRose et al., 2022):

$$\text{MSE} = \text{Bias}^2 + \text{Variance}$$


In [ ]:
analysis = zne.analyze_bias_variance(
    circuit=circuit, qubit=0,
    ideal_value=ideal_exp_z, num_trials=30
)

# Results table
print(f'{"Metric":<15} {"Unmitigated":>14} {"Mitigated":>14}')
print('=' * 45)
for metric in ['mean', 'bias', 'variance', 'mse']:
    u = analysis['unmitigated'][metric]
    m = analysis['mitigated'][metric]
    print(f'{metric.upper():<15} {u:>14.6f} {m:>14.6f}')

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

metrics = ['bias', 'variance', 'mse']
colors = ['#3498db', '#9b59b6']
for ax, metric in zip(axes, metrics):
    vals = [abs(analysis['unmitigated'][metric]), abs(analysis['mitigated'][metric])]
    bars = ax.bar(['Unmitigated', 'Mitigated'], vals, color=colors, edgecolor='white', linewidth=2)
    ax.set_title(metric.upper(), fontweight='bold')
    ax.set_ylabel('Value')
    for bar, v in zip(bars, vals):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                f'{v:.4f}', ha='center', va='bottom', fontsize=10)

plt.suptitle('Bias-Variance Tradeoff in ZNE', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()


## 9. Performance Benchmarks

QMitigate's C++ core with OpenMP delivers significant performance advantages.


In [ ]:
from qmitigate import scaling_benchmark

print('Performance Scaling Benchmark')
print('=' * 60)
results = scaling_benchmark(max_qubits=20)


In [ ]:
if results:
    qubits = [r['num_qubits'] for r in results]
    times = [r['mean_time_seconds'] for r in results]
    mem = [r['state_vector_size_mb'] for r in results]

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    ax1.semilogy(qubits, times, 'o-', color='#3498db', linewidth=2, markersize=10)
    ax1.set_xlabel('Number of Qubits')
    ax1.set_ylabel('Execution Time (s)')
    ax1.set_title('Simulation Time Scaling', fontweight='bold')
    ax1.grid(True, alpha=0.3)

    ax2.semilogy(qubits, mem, 's-', color='#e74c3c', linewidth=2, markersize=10)
    ax2.set_xlabel('Number of Qubits')
    ax2.set_ylabel('State Vector Size (MB)')
    ax2.set_title('Memory Requirements', fontweight='bold')
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('performance_scaling.png', dpi=150, bbox_inches='tight')
    plt.show()
    print('✓ Saved: performance_scaling.png')


## 10. Advanced Examples

### 10.1 ZNE for Multi-Qubit Entangled States

Applying ZNE to a GHZ preparation circuit — a stress test for error mitigation.


In [ ]:
# GHZ circuit with ZNE
ghz_circuit = Circuit(4)
ghz_circuit.h(0)
for q in range(3):
    ghz_circuit.cnot(q, q + 1)

ideal_state = Simulator().run(ghz_circuit)
ideal_z = ideal_state.expectation_Z(0)

# Test multiple error rates with ZNE
test_rates = [0.005, 0.01, 0.02, 0.05]
raw_errors = []
zne_errors = []

for rate in test_rates:
    noise = create_depolarizing_noise_model(rate)
    noisy_sim = Simulator(noise)
    
    # Raw noisy
    raw_vals = [noisy_sim.run(ghz_circuit).expectation_Z(0) for _ in range(50)]
    raw_errors.append(abs(ideal_z - np.mean(raw_vals)))
    
    # ZNE mitigated
    zne = ZeroNoiseExtrapolator(noisy_sim, scale_factors=[1, 3, 5], shots_per_scale=30)
    mit_val, _ = zne.mitigate_expectation_Z(ghz_circuit, qubit=0)
    zne_errors.append(abs(ideal_z - mit_val))

# Plot comparison
fig, ax = plt.subplots(figsize=(10, 5))
x = np.arange(len(test_rates))
w = 0.35
ax.bar(x - w/2, raw_errors, w, label='Raw (Noisy)', color='#e74c3c', edgecolor='white')
ax.bar(x + w/2, zne_errors, w, label='ZNE Mitigated', color='#2ecc71', edgecolor='white')
ax.set_xticks(x)
ax.set_xticklabels([f'{r*100:.1f}%' for r in test_rates])
ax.set_xlabel('Gate Error Rate')
ax.set_ylabel('|Error from Ideal|')
ax.set_title('ZNE Effectiveness Across Error Rates (4-Qubit GHZ)', fontweight='bold')
ax.legend()
plt.tight_layout()
plt.show()


## 11. API Reference Summary

### Core Classes

| Class | Description | Key Methods |
|-------|------------|-------------|
| `QuantumState(n)` | N-qubit state vector | `expectation_Z()`, `get_probabilities()`, `fidelity()`, `norm()` |
| `Circuit(n)` | Quantum circuit builder | `h()`, `x()`, `cnot()`, `rx()`, `ry()`, `rz()`, `depth()` |
| `Simulator(noise?)` | State-vector simulator | `run(circuit)`, `seed()`, `set_noise_model()` |
| `ZeroNoiseExtrapolator` | ZNE engine | `mitigate_expectation_Z()`, `analyze_bias_variance()` |

### Factory Functions

| Function | Description |
|----------|------------|
| `create_bell_state()` | Create $(|00\rangle + |11\rangle)/\sqrt{2}$ |
| `create_ghz_state(n)` | Create $(|00\ldots0\rangle + |11\ldots1\rangle)/\sqrt{2}$ |
| `create_w_state(n)` | Create W state |
| `create_depolarizing_noise_model(p)` | Depolarizing channel with error rate $p$ |
| `fold_circuit_global(circuit, s)` | Digital folding at scale factor $s$ |

### Extrapolation Functions

| Function | Method | Returns |
|----------|--------|--------|
| `richardson_extrapolate(scales, values)` | Polynomial fit | Zero-noise estimate |
| `linear_extrapolate(scales, values)` | Linear fit (first 2 points) | (estimate, slope) |
| `exponential_extrapolate(scales, values)` | $a \cdot e^{-bx} + c$ fit | (estimate, params) |


---

<div align='center'>

**Built with ❤️ for the Quantum Computing Research Community**

*Demonstrating the intersection of HPC Systems, Quantum Physics, and Research Engineering*

</div>
